In [3]:
siteUrl = "http://www.trade-a-plane.com/filtered/search"
siteUrl = "?".join([siteUrl, "s-type=aircraft"])
siteUrl = "&".join([siteUrl, "s-keyword-search="])
print(siteUrl)

proxies = {'http': 'http://<<USERID>>:<<PASSWORD>>@23.19.51.245:40791/'}

http://www.trade-a-plane.com/filtered/search?s-type=aircraft&s-keyword-search=


In [8]:
import requests
from bs4 import BeautifulSoup

# Retrieve the page with tag results and set it up to be scraped
sitePage = requests.get(url=siteUrl, proxies=proxies)
siteSoup = BeautifulSoup(sitePage.content, 'lxml')

#<a href="/search?s-type=aircraft&amp;s-keyword-search=&amp;s-page=137"> &gt;&gt; </a>
siteLinks = siteSoup.select('a[href*=s-page]')
#print(siteLinks)

# Get the max page link
max_page = 0
if len(siteLinks):
    max_page = int(siteLinks[-1]['href'].split('=')[-1])
print(max_page)

137


In [18]:
from random import randint

i = randint(1, max_page)
tagUrl = "&".join([siteUrl, "s-page="+str(i)])
print(tagUrl)

tagPage = requests.get(url=tagUrl, proxies=proxies)
tagSoup = BeautifulSoup(tagPage.content, 'lxml')

# Get the title of the page to prove we are progressing
tagTitle = tagSoup.select("head > title")
titleString = tagTitle[0].get_text().strip()
print(titleString)

http://www.trade-a-plane.com/filtered/search?s-type=aircraft&s-keyword-search=&s-page=44
Aircraft For Sale - Used & New 1076 - 1100


In [21]:
#<span itemprop="telephone">(260) 238-5828 </span>
phoneLinks = tagSoup.select('span[itemprop="telephone"]')

phoneLink = phoneLinks[randint(0,len(phoneLinks)-1)]

In [25]:
import re

phone = phoneLink.get_text().strip()
print(phone)

if len(phone):

    ##result_listing > div > div.result_info > div > p.center > span
    phoneSoup = phoneLink.parent.parent.parent.parent.parent
    #print(phoneSoup)

    #<a href="/company-search/details?id=46862" title="Aircraft Canada" itemprop="name">Aircraft Canada</a>
    seller = phoneSoup.select('a[itemprop="name"]')
    if(len(seller)):
        seller = seller[0].get_text().strip()
        seller = re.sub(r'\s+', ' ', seller)
    else:
        seller = "Unknown"
    print(seller)

    model = phoneSoup.select('#title')
    if(len(model)):
        model = model[0].get_text().strip()
        model = re.sub(r'\s+', ' ', model)
    else:
        model = "Unknown"
    print(model)

    price = phoneSoup.select('ul.result_header > li.price')
    if(len(price)):
        price = price[0].get_text().strip()
    else:
        price = "Unknown"
    print(price)

(403) 547-1024  Main Ofc
Aircraft Canada
1988 BEECHCRAFT KING AIR B200 (For Sale)
$1,295,000 USD


In [16]:
for i in range(1, max_page+1):
    tagUrl = "&".join([siteUrl, "s-page="+str(i)])

    tagPage = requests.get(url=tagUrl, proxies=proxies)
    tagSoup = BeautifulSoup(tagPage.content, 'lxml')
    
    phoneLinks = tagSoup.select('span[itemprop="telephone"]')
    for phoneLink in phoneLinks:
        
        phone = phoneLink.get_text().strip()
        if len(phone):

            phoneSoup = phoneLink.parent.parent.parent.parent.parent
            
            seller = phoneSoup.select('a[itemprop="name"]')
            if(len(seller)):
                seller = seller[0].get_text().strip()
                seller = re.sub(r'\s+', ' ', seller)
            else:
                seller = "Unknown"

            model = phoneSoup.select('#title')
            if(len(model)):
                model = model[0].get_text().strip()
                model = re.sub(r'\s+', ' ', model)
            else:
                model = "Unknown"

            price = phoneSoup.select('ul.result_header > li.price')
            if(len(price)):
                price = price[0].get_text().strip()
            else:
                price = "Unknown"
            
            bigFile = open('trade-a-plane.txt', 'a', encoding='utf-8')
            bigFile.write(seller + ': ' + model + '\t' + price + '\t' + phone + '\n')
            bigFile.close()